In [1]:
from evdev import InputDevice, ecodes
device = InputDevice('/dev/input/event0')   # Events for Xbox One S Wireless Controller

In [2]:
# Establish connection with server
import os, ssl, socket
server_ip = 'yzlab3.chem.nyu.edu'
ctrl_port = 54322  
server_cert = 'secret/server.crt'
client_cert = 'secret/driver.crt'
client_key  = 'secret/driver.key'

context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert)
context.load_cert_chain(certfile=client_cert, keyfile=client_key)

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sc = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
sc.connect((server_ip, ctrl_port))

In [ ]:
import struct
for event in device.read_loop(): 
    if event.type in [ecodes.EV_ABS, ecodes.EV_KEY]:  
        #print(event.code, event.value)
        sc.send(struct.pack('<Hi', event.code, event.value)) # Send code and value to server